<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import project files files from github 


In [1]:
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

!pip3 install import_ipynb

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 289 (delta 8), reused 0 (delta 0), pack-reused 274
Receiving objects: 100% (289/289), 159.51 KiB | 222.00 KiB/s, done.
Resolving deltas: 100% (152/152), done.
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=a94bcf696132ca58cea523dfbcc160cbd610f4104767d5941e69b562179c85e5
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


import libraries


In [2]:
import torch
import os
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
from torchvision import transforms
import numpy as np
import import_ipynb
import copy
# project classes --------------------------------------------------------------
from IncrementalLearning.cifar100 import ilCIFAR100
from resnet_cifar import resnet32
# from info_recorder.info_log import InfoLog
# from models_benchmark.benchmark import Benchmark

importing Jupyter notebook from /content/IncrementalLearning/cifar100.ipynb


Extracting ./data/cifar-100-python.tar.gz to ./data


Initial setup

In [6]:
# Define params

NUM_RUN = 2
SEEDS = [1,2,3]
NUM_EPOCHS = 70
BATCH_SIZE = 128
CLASSES_PER_BATCH = 10
DEVICE = 'cuda'
NUM_BATCHES=10
WEIGHT_DECAY = 1e-5
LR = 2
MOMENTUM = 0.9
MILESTONES = [49, 63]
GAMMA = 0.2
randomseed=981
SAVINGS_DIR = "savings"
# Create savings folder
try:
    os.mkdir(SAVINGS_DIR)
except FileExistsError:
    pass

Prepare dataloders for each train, val test subset of the original dataset

FINE TUNING

In [7]:
def trainmodel(model,trainloader,valloader,learningrate,weightdecay,numepochs,milestones,gamma,criterion,optimizer,scheduler,dictionary):
  model=model.to(DEVICE)
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  for epoch in range(numepochs):
    #print(epoch)
    for phase in ['train','val']:
      if phase =='train':
        model.train()
        loader = trainloader
      else:
        model.eval()
        loader = valloader
      running_loss=0.0
      running_corrects=0
      num_elements=0
      for inputs,labels in loader:
        labels = torch.tensor([torch.tensor(diz[c.item()]) for c in labels])
        num_elements+=len(inputs)
        inputs=inputs.to(DEVICE)
        labels=labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.set_grad_enabled(phase == 'train'):
          outputs=model(inputs)
          labels_encoded=torch.eye(100)[labels] #CAMBIARE ONE_HOT
          labels_encoded=labels_encoded.to(DEVICE)
          _,preds=torch.max(outputs,1)
          loss=criterion(outputs,labels_encoded)
          if phase=='train':
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss / num_elements
      epoch_acc = running_corrects.double() / num_elements
      
            # deep copy the model
      if phase == 'val' and epoch_acc > best_acc:
          best_acc = epoch_acc
          best_model_wts = copy.deepcopy(model.state_dict())
          #print(best_acc)
    # load best model weights
  model.load_state_dict(best_model_wts)
  return model


In [8]:
original_training_set = ilCIFAR100(CLASSES_PER_BATCH,randomseed)
original_test_set = ilCIFAR100(CLASSES_PER_BATCH,randomseed, train=False)
train_indices, val_indices = original_training_set.get_train_val(0.1)
test_indices = original_test_set.get_batch_indexes()
current_test_indexes=[]
model=resnet32(num_classes=100)
diz = original_training_set.get_dict()
acc=[]
criterion=nn.BCEWithLogitsLoss()
for i in range(NUM_BATCHES):
  optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=MILESTONES, gamma=GAMMA)
  print(i)
  train_dataset=Subset(original_training_set,train_indices[i])
  val_dataset=Subset(original_training_set,val_indices[i])
  current_test_indexes+=test_indices[i].tolist()
  test_dataset=Subset(original_test_set,current_test_indexes)
  train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
  test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
  net=trainmodel(model,train_dataloader,val_dataloader,LR,WEIGHT_DECAY,NUM_EPOCHS,MILESTONES,GAMMA,criterion,optimizer,scheduler,diz)
  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  net.train(False) # Set Network to evaluation mode
  running_corrects = 0
  y_true = []
  y_pred = []
  for inputs, labels in test_dataloader:
      labels = torch.tensor([torch.tensor(diz[c.item()]) for c in labels])
      inputs = inputs.to(DEVICE)
      labels = labels.to(DEVICE)
      outputs = net(inputs)
      _, preds = torch.max(outputs.data, 1)
      running_corrects += torch.sum(preds == labels.data).data.item()
      labels=labels.detach().cpu().numpy()
      labels=labels.tolist()
      y_true.extend(labels)
      preds=preds.detach().cpu().numpy()
      y_pred_tmp = [p for p in preds]
      y_pred.extend(y_pred_tmp)

  # Calculate Accuracy
  accuracy = running_corrects / float(len(test_dataloader.dataset))
  print('Test Accuracy: %.2f' % (100.0 * accuracy))
  print('-' * 80)
  acc.append(accuracy)

Files already downloaded and verified
Files already downloaded and verified
0
Test Accuracy: 85.30
--------------------------------------------------------------------------------
1
Test Accuracy: 42.85
--------------------------------------------------------------------------------
2
Test Accuracy: 30.57
--------------------------------------------------------------------------------
3
Test Accuracy: 21.77
--------------------------------------------------------------------------------
4
Test Accuracy: 18.20
--------------------------------------------------------------------------------
5
Test Accuracy: 14.85
--------------------------------------------------------------------------------
6
Test Accuracy: 12.53
--------------------------------------------------------------------------------
7
Test Accuracy: 11.07
--------------------------------------------------------------------------------
8
Test Accuracy: 9.74
----------------------------------------------------------------------